<h1>Transfer Learning - InceptionV3<h1>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/BTech_Project/preprocessed_v2/Training'
test_dir = '/content/drive/MyDrive/Colab Notebooks/BTech_Project/preprocessed_v2/Testing'

In [ ]:
#augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    validation_split=0.15
)

#no augmentation for validation and test sets
val_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.15)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

#load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

#load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 4687 images belonging to 4 classes.
Found 826 images belonging to 4 classes.
Found 1212 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.regularizers import l2

#load the pre-trained InceptionV3 base model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

#add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)

#combine base model and custom layers into one model
model = Model(inputs=base_model.input, outputs=x)

#freezing the base model layers initially
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
#callbacks to adjust learning rate and prevent overfitting
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[reduce_lr, early_stopping]
)

model.save('InceptionV3_initial.keras')

Epoch 1/15
147/147 [==============================] - 1317s 9s/step - loss: 2.1902 - accuracy: 0.6475 - val_loss: 1.9457 - val_accuracy: 0.7058 - lr: 1.0000e-04
Epoch 2/15
147/147 [==============================] - 145s 984ms/step - loss: 1.8041 - accuracy: 0.7862 - val_loss: 1.8344 - val_accuracy: 0.7603 - lr: 1.0000e-04
Epoch 3/15
147/147 [==============================] - 146s 991ms/step - loss: 1.6491 - accuracy: 0.8133 - val_loss: 1.7421 - val_accuracy: 0.7772 - lr: 1.0000e-04
Epoch 4/15
147/147 [==============================] - 146s 996ms/step - loss: 1.5520 - accuracy: 0.8291 - val_loss: 1.6079 - val_accuracy: 0.7833 - lr: 1.0000e-04
Epoch 5/15
147/147 [==============================] - 145s 986ms/step - loss: 1.4730 - accuracy: 0.8370 - val_loss: 1.5603 - val_accuracy: 0.7821 - lr: 1.0000e-04
Epoch 6/15
147/147 [==============================] - 144s 979ms/step - loss: 1.3889 - accuracy: 0.8481 - val_loss: 1.5613 - val_accuracy: 0.7748 - lr: 1.0000e-04
Epoch 7/15
147/147 [====

In [1]:
#unfreeze the last layers of the base model for fine-tuning
for layer in base_model.layers[-90:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

#fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[reduce_lr, early_stopping]
)

model.save('InceptionV3_finetuned.keras')

Epoch 1/15
147/147 [==============================] - 169s 1s/step - loss: 1.0575 - accuracy: 0.8385 - val_loss: 1.2463 - val_accuracy: 0.7978 - lr: 1.0000e-05
Epoch 2/15
147/147 [==============================] - 161s 1s/step - loss: 0.9513 - accuracy: 0.8505 - val_loss: 1.0454 - val_accuracy: 0.8475 - lr: 1.0000e-05
Epoch 3/15
147/147 [==============================] - 162s 1s/step - loss: 0.8816 - accuracy: 0.8780 - val_loss: 0.9967 - val_accuracy: 0.8489 - lr: 1.0000e-05
Epoch 4/15
147/147 [==============================] - 162s 1s/step - loss: 0.8331 - accuracy: 0.8923 - val_loss: 0.9781 - val_accuracy: 0.8432 - lr: 1.0000e-05
Epoch 5/15
147/147 [==============================] - 160s 1s/step - loss: 0.7963 - accuracy: 0.8989 - val_loss: 0.9589 - val_accuracy: 0.8553 - lr: 1.0000e-05
Epoch 6/15
147/147 [==============================] - 160s 1s/step - loss: 0.7687 - accuracy: 0.9094 - val_loss: 0.9198 - val_accuracy: 0.8589 - lr: 1.0000e-05
Epoch 7/15
147/147 [====================

In [2]:
print(f"Training accuracy: {fine_tune_history.history['accuracy'][-1]:.4f}")
print(f"Validation accuracy: {fine_tune_history.history['val_accuracy'][-1]:.4f}")

Training accuracy: 0.9491
Validation accuracy: 0.8905


In [3]:
#evaluating the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

#calculating F1 score
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f"F1 Score: {f1}")

38/38 [==============================] - 367s 10s/step - loss: 0.7067 - accuracy: 0.9348
Test Accuracy: 0.9157449096679688
38/38 [==============================] - 23s 592ms/step
F1 Score: 0.9084593486386093
